In [2]:
# Importing Libraries
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import mlflow

In [3]:
# Creating MLFlow Experiment

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
experiment_name= "classic_evaluation"
mlflow.set_experiment(experiment_name)

2024/12/15 11:40:43 INFO mlflow.tracking.fluent: Experiment with name 'classic_evaluation' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/159889069568961193', creation_time=1734243043122, experiment_id='159889069568961193', last_update_time=1734243043122, lifecycle_stage='active', name='classic_evaluation', tags={}>

In [4]:
# Creating a sample dataset

x, y= make_classification(n_samples=1000, n_features=5, n_classes=2, random_state=42)

x_df=pd.DataFrame(x, columns=[f"feature_{i+1}"for i in range(x.shape[1])])
y_df=pd.DataFrame(y, columns=['target'])
x_train, x_test,y_train,y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=42)


In [5]:
# Random Forest Classifier
rfc= RandomForestClassifier(n_estimators=1)
rfc.fit(x_train, y_train)
predictions=rfc.predict(x_test)

c:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [7]:
eval_df= pd.DataFrame({
    "target": y_test.to_numpy().flatten(),
    "predictions": predictions

})
print(eval_df.head())

   target  predictions
0       1            1
1       1            1
2       1            1
3       1            1
4       0            1


In [8]:
#using the eval_df of evaluating model

result = mlflow.evaluate(
    data= eval_df,
    model_type="classifier",
    targets="target",
    predictions="predictions"
)

c:\Users\admin\anaconda3\Lib\site-packages\mlflow\types\utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/12/15 13:18:52 WARNING mlflow.models.evaluation.evaluators.classifier: According to the evaluation dataset label values, the model type looks like None, but you specified model type 'classifier'. Please verify that you set the `model_type` 

🏃 View run fortunate-yak-543 at: http://127.0.0.1:5000/#/experiments/159889069568961193/runs/b37704952f8b4e7889ef21d4ad211751
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/159889069568961193


<Figure size 1050x700 with 0 Axes>

In [9]:
# Evaluating the model using estimator

# Using model as a function


def random_forest_clf(model_input):
    return rfc.predict(model_input)


In [10]:
eval_df_for_model= x_test.copy()
eval_df_for_model["target"]=y_test

In [11]:
eval_df_for_model.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,target
521,1.250932,-1.064291,-2.238231,-0.914547,1.261286,1
737,-0.196283,0.190820,-0.243384,0.154804,-0.256094,1
740,2.659138,-0.265773,1.072978,-0.996758,-2.195564,1
660,0.087778,-0.021011,-0.667780,-0.038709,-0.042586,1
411,-0.662457,0.741043,-0.358340,0.568499,-1.101298,0


In [12]:
result= mlflow.evaluate(
    model=random_forest_clf,
    data=eval_df_for_model,
    model_type="classifier",
    targets="target",
    predictions = "predictions"

)

c:\Users\admin\anaconda3\Lib\site-packages\mlflow\types\utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/12/15 13:25:49 WARNING mlflow.models.evaluation.evaluators.classifier: According to the evaluation dataset label values, the model type looks like None, but you specified model type 'classifier'. Please verify that you set the `model_type` 

🏃 View run bouncy-swan-560 at: http://127.0.0.1:5000/#/experiments/159889069568961193/runs/93394f6b763043649261d6cc67b0c3bc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/159889069568961193


In [13]:
result.metrics

{'true_negatives': 84,
 'false_positives': 13,
 'false_negatives': 9,
 'true_positives': 94,
 'example_count': 200,
 'accuracy_score': 0.89,
 'recall_score': 0.912621359223301,
 'precision_score': 0.8785046728971962,
 'f1_score': 0.8952380952380953}